# Conexión con API de Google Maps - Cálculo de distancias entre estaciones de carga de vehículo eléctrico en Cataluña

El objetivo de este notebook es calcular la distancia en coche, a través de la red de carreteras existente, entre los puntos de recarga de vehículo eléctrico presentes en Cataluña. La localización exacta de cada punto de recarga en Cataluña, está disponible como información abierta en el siguiente enlace: https://datos.gob.es/es/catalogo/a09002970-estaciones-de-recarga-para-vehiculos-electricos-en-cataluna

A la hora de calcular la distancia entre puntos, es importante tener en cuenta que se quiere medir la autonomía del coche eléctrico y su consum. No sería realista para esta casuística calcular simplemente la distancia en línea entre dos puntos del globo terráqueo (lo cual se puede calcular matemáticamente mediante la fórmula del semiverseno: https://es.wikipedia.org/wiki/F%C3%B3rmula_del_semiverseno) 

Para poder calcular adecuadamente la distancia real entre pares de puntos y sus coordenadas (LATITUD,LONGITUD) se hará uso de la API disponible de Google Maps.

Se importan las librerías necesarias, y se carga el fichero .csv con la información de cada uno de los puntos de carga. Se define también la clave API personal para poder conectar apropiadamente con los servicios de Google.

In [134]:
import requests
import json
import googlemaps
from datetime import datetime
from itertools import combinations
import pandas as pd
import time
api_key = "YOUR_API_KEY"

df_pc_cat = pd.read_csv(r'C:\Users\Usuario\Desktop\Master UOC\Master ciencia de datos\TFM\Estacions_de_rec_rrega_per_a_vehicle_el_ctric_a_Catalunya.csv',
                        delimiter=";")

Se muestra la estructura del fichero de puntos de recarga, así como las dimensiones del mismo.

In [96]:
df_pc_cat.head()

,PROMOTOR-GESTOR,ACCES,TIPUS VELOCITAT,TIPUS CONNEXIÓ,LATITUD,LONGITUD,DESIGNACIÓ-DESCRIPTIVA,POTENCIA,TIPUS DE CORRENT,INDENTIFICADOR,ADREÇA,PROVINCIA,CODIPROV,MUNICIPI,NPLACES ESTACIÓ,TIPUS VEHICLE,Columna amb georeferència
0,CC ALDI,APARCAMENT CC,NORMAL,MENNEKES.F,41.321165,2.027702,CC ALDI,NaN,NaN,NaN,"Carretera Barcelona, 84",Barcelona,8,Viladecans,4,cotxe,POINT (2.027702 41.321165)
1,NaN,NaN,semiRAPID,NaN,40.807620,0.520070,EdRsR Tortosa (AC22kW) PIRVEC-24,22.0,AC,PIRVEC-24,"Carrer d'Alcanyís, 9",Tarragona,43,Tortosa,NaN,NaN,POINT (0.52007 40.80762)
2,HOTEL NH CONSTANZA,HOTEL,NORMAL,Schuko,41.384550,2.137671,HOTEL NH NUMANCIA,NaN,NaN,NaN,"Carrer de Numància, 74",Barcelona,8,Barcelona,1,NaN,POINT (2.1376712999999654 41.38455)
3,B:SM Serveis Municipals,APARCAMENT PUBLIC,NORMAL,MENNEKES.F+Schuko,41.394009,2.115265,B:SM 20 - Marques de Mulhacen,NaN,NaN,BSM015,"Carrer de Marquès de Mulhacén, 49-51",Barcelona,8,Barcelona,6+2,cotxe i moto,POINT (2.115265 41.394009)
4,B:SM Serveis Municipals,APARCAMENT PUBLIC,NORMAL,Schuko,41.404700,2.189600,B:SM 24 - Ona Glòries,NaN,NaN,BSM037,"Carrer de la Ciutat de Granada, 173-175",Barcelona,8,Barcelona,4,cotxe,POINT (2.1896 41.4047)


In [138]:
shape = df_pc_cat.shape
print('El numero de filas del dataset es: %d' % (shape[0]))
print('El numero de columnas del dataset es: %d' % (shape[1]))

El numero de filas del dataset es: 418
El numero de columnas del dataset es: 17


Como se puede comprobar, cada registro muestra el detalle de cada uno de los puntos de recarga disponibles (localización, designación, tipo vehículo, potencia). Existen 418 puntos en el registro.

La distancia en coche debe calcularse entre todos los pares (LATITUD,LONGITUD) presentes en el dataset, es decir, entre todas las estaciones de carga presentes en Cataluña a fecha de última actualización del fichero (Mayo de 2021). Se genera por tanto una lista de tuplas con todos los pares de puntos presentes en el dataset.

In [97]:
lista_coord = []
for index, row in df_pc_cat.iterrows():
    elem = (row['LATITUD'],row['LONGITUD'])
    lista_coord.append(elem)

A continuación es necesario obtener una lista con todas las combinaciones de pares de puntos posibles. Es decir, se calcularán todas las posibles combinaciones sin repetición en conjuntos de 2 elementos. La librería itertools dispone de una función específica para esta tarea: la función **combinations**. De esta forma se genera una lista de tuplas con la información anterior.

In [117]:
lista_combo = []
for combo in combinations(lista_coord, 2):
    lista_combo.append(combo)

In [118]:
len(lista_combo)

87153

Existen 87153 combinaciones de 2 elementos sin repetición. Se muestra a continuación uno de los elementos para comprobar su estructura.

In [137]:
lista_combo[1]

((41.321165, 2.027702), (41.38455, 2.1376713))

A continuación se realizará la llamada a la API de Google Maps. Se iterará para cada elemento de la tupla, siendo cada elemento un par latitud, longitud de dos puntos geográficos. En cada iteración se obtiene la distancia de conducción con vehículo terrestre y la duración del trayecto, a fecha 09/12/2021 a las 19:00 pm. Se ha seleccionado esta hora para evitar más o menos tráfico del habitual por festivos, horas punta, etc. Se aplica un retardo de 1 segundo por cada 100 iteraciones para evitar bloqueos de la API.

In [129]:
lista_distancia = []
lista_duracion = []
i = 0
#Se itera para cada tupla de la lista. En cada iteración se llama a la API para cada par de puntos contenidos en la tupla.
#Se calculan distancias y duraciones de trayecto en el tiempo de la ejecución del algoritmo.
for index, tuple in enumerate(lista_combo):
    gmaps = googlemaps.Client(key=api_key)
    now = datetime.now()
    direcciones = gmaps.directions(tuple[0],
                                     tuple[1],
                                     mode="driving",
                                     departure_time=now
                                    )
    lista_distancia.append(direcciones[0]['legs'][0]['distance']['text'])
    lista_duracion.append(direcciones[0]['legs'][0]['duration']['text'])
    #Se añade un pequeño retardo de un segundo cada 100 iteraciones, para evitar saturar la API.
    i = i + 1
    if i % 100 == 0:
        time.sleep(1)

Se crea una nueva tupla con la distancia y la duración para cada combinación de puntos  Se unen posteriormente las dos tuplas en un Dataframe, el cual se limpia de columnas innecesarias.

In [130]:
data_tuples = list(zip(lista_distancia,lista_duracion))

In [141]:
df_salida = pd.DataFrame(data_tuples, columns=['distancia','duracion'])
destinos = pd.DataFrame(lista_combo, columns=['inicio','final'])
df_destinos = df_destinos.iloc[:, [1,2]]
df_grafo = pd.concat([df_destinos, df_distancias], axis=1, join="inner")
df_grafo = df_grafo.iloc[:, [0,1,3,4]]

Se muestra a continuación la estructura del Dataframe construido

In [142]:
df_grafo

,inicio,final,distancia,duracion
0,"(41.321165, 2.027702)","(40.80762, 0.52007)",164 km,1 hour 42 mins
1,"(41.321165, 2.027702)","(41.38455, 2.1376713)",15.1 km,19 mins
2,"(41.321165, 2.027702)","(41.394009000000004, 2.115265)",13.8 km,16 mins
3,"(41.321165, 2.027702)","(41.4047, 2.1896)",22.5 km,27 mins
4,"(41.321165, 2.027702)","(41.66519, 1.86011)",52.1 km,42 mins
...,...,...,...,...
87148,"(41.613690000000005, 2.3454900000000003)","(41.389255, 2.161321)",41.7 km,39 mins
87149,"(41.613690000000005, 2.3454900000000003)","(41.56064, 2.0059)",37.7 km,36 mins
87150,"(41.372299, 2.153134)","(41.389255, 2.161321)",40.8 km,33 mins
87151,"(41.372299, 2.153134)","(41.56064, 2.0059)",3.3 km,13 mins


Por último, se aplicará un proceso similar al anterior para obtener las designaciones de cada uno de los puntos origen y destino, de forma que se puedan identificar a posteriori.

In [143]:
lista_denom= []
for index, row in df_pc_cat.iterrows():
    elem = row['DESIGNACIÓ-DESCRIPTIVA']
    lista_denom.append(elem)

lista_combo_denom = []
for combo_denom in combinations(lista_denom, 2):  # 2 for pairs, 3 for triplets, etc
    lista_combo_denom.append(combo_denom)

In [144]:
df_denom = pd.DataFrame(lista_combo_denom, columns=['denom_ini','denom_fin'])
df_grafo_desc = pd.concat([df_grafo,df_denom],axis=1, join="inner")

Se muestra el resultado con las nuevas columnas añadidas (denom_ini y denom_fin) y se exporta a un fichero csv para posteriores análisis.

In [145]:
df_grafo_desc

,inicio,final,distancia,duracion,denom_ini,denom_fin
0,"(41.321165, 2.027702)","(40.80762, 0.52007)",164 km,1 hour 42 mins,CC ALDI,EdRsR Tortosa (AC22kW) PIRVEC-24
1,"(41.321165, 2.027702)","(41.38455, 2.1376713)",15.1 km,19 mins,CC ALDI,HOTEL NH NUMANCIA
2,"(41.321165, 2.027702)","(41.394009000000004, 2.115265)",13.8 km,16 mins,CC ALDI,B:SM 20 - Marques de Mulhacen
3,"(41.321165, 2.027702)","(41.4047, 2.1896)",22.5 km,27 mins,CC ALDI,B:SM 24 - Ona Glòries
4,"(41.321165, 2.027702)","(41.66519, 1.86011)",52.1 km,42 mins,CC ALDI,EdRsR Sant Vicenç de Castellet (AC22kW)
...,...,...,...,...,...,...
87148,"(41.613690000000005, 2.3454900000000003)","(41.389255, 2.161321)",41.7 km,39 mins,CC La Roca Village Aparcament (11kW),28 AjBCN-Endesa-Aragó-Pl.Letamendi
87149,"(41.613690000000005, 2.3454900000000003)","(41.56064, 2.0059)",37.7 km,36 mins,CC La Roca Village Aparcament (11kW),Aparcament Plaça Progrés
87150,"(41.372299, 2.153134)","(41.389255, 2.161321)",40.8 km,33 mins,B:SM 35 - Rius i Taulet,28 AjBCN-Endesa-Aragó-Pl.Letamendi
87151,"(41.372299, 2.153134)","(41.56064, 2.0059)",3.3 km,13 mins,B:SM 35 - Rius i Taulet,Aparcament Plaça Progrés


In [146]:
df_grafo_desc.to_csv('df_grafo_desc.csv',index=False)